### Objective

To create stories using out of box embeddings and on candidate articles

In [1]:
parent_folder = '/Users/username/Desktop/ML/Recommendations/arcane/'
from hydra import compose, initialize
import os

import xml.etree.ElementTree as ET

tree = ET.parse('../../conf/application.run.xml')
root = tree.getroot()

envs_element = root.find('./configuration/envs')
for variable in envs_element.findall('env'):
    name = variable.get('name')
    value = variable.get('value')
    os.environ[name] = value

import sys
sys.path.append('/Users/username/Desktop/ML/Recommendations/arcane/')

In [2]:
import numpy as np
import pandas as pd
import json

from sql.articles.MongoDBArticle import MongoDBArticle

from sql.PostgresDatabaseOperation import PostgresDatabaseOperation

In [3]:
with PostgresDatabaseOperation() as cursor:
    get_sql = """SELECT DISTINCT e.article_id, embedding
                 FROM embeddings e 
                 INNER JOIN candidate_articles laa
                 ON e.article_id = laa.article_id
                 WHERE laa.content_type = 'ARTICLE'
                 """
    cursor.execute(get_sql)
    results = cursor.fetchall()

In [4]:
embeddings = {x[0]: [np.float16(y) for y in json.loads(x[1])] for x in results}

In [5]:
len(embeddings)

6863

In [6]:
candidate_article_ids = list(embeddings.keys())

In [7]:
articles = MongoDBArticle.fetch_documents_by_ids(candidate_article_ids)

In [8]:
len(articles)

6863

In [9]:
missing_text = [article for article in articles if 'cleaned_text' not in article]

In [10]:
len(missing_text)

0

2024-02-13 11:41:31,975| ERROR   | Could not establish connection from local ('127.0.0.1', 3431) to remote ('insight-beta-article-db.cluster-chmaiagjipqy.ap-south-1.docdb.amazonaws.com', 27017) side of the tunnel: open new channel ssh error: Timeout opening channel.
2024-02-13 11:41:52,095| ERROR   | Could not establish connection from local ('127.0.0.1', 3431) to remote ('insight-beta-article-db.cluster-chmaiagjipqy.ap-south-1.docdb.amazonaws.com', 27017) side of the tunnel: open new channel ssh error: Timeout opening channel.
2024-02-13 11:42:12,229| ERROR   | Could not establish connection from local ('127.0.0.1', 3431) to remote ('insight-beta-article-db.cluster-chmaiagjipqy.ap-south-1.docdb.amazonaws.com', 27017) side of the tunnel: open new channel ssh error: Timeout opening channel.
2024-02-13 11:42:28,910| ERROR   | Socket exception: Operation timed out (60)
2024-02-13 11:42:28,913| ERROR   | Could not establish connection from local ('127.0.0.1', 3431) to remote ('insight-beta

In [40]:
article_documents = [articles[i]['cleaned_text'] for i in range(len(candidate_article_ids)) if str(articles[i]['_id']) == candidate_article_ids[i]]

KeyError: 'cleaned_text'

2024-02-13 10:59:45,400| ERROR   | Could not establish connection from local ('127.0.0.1', 3892) to remote ('insight-beta-article-db.cluster-chmaiagjipqy.ap-south-1.docdb.amazonaws.com', 27017) side of the tunnel: open new channel ssh error: SSH session not active


In [37]:
len(article_documents)

7433

In [39]:
article_documents[0]

'Stock market scams and how to avoid them - Pump and Dump – Varsity by Zerodha'

2024-02-13 10:59:35,312| ERROR   | Could not establish connection from local ('127.0.0.1', 3892) to remote ('insight-beta-article-db.cluster-chmaiagjipqy.ap-south-1.docdb.amazonaws.com', 27017) side of the tunnel: open new channel ssh error: SSH session not active


### Fitting bertopic

In [15]:
from umap import UMAP
from hdbscan import HDBSCAN
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer

2024-02-13 10:50:51,568| ERROR   | Could not establish connection from local ('127.0.0.1', 3892) to remote ('insight-beta-article-db.cluster-chmaiagjipqy.ap-south-1.docdb.amazonaws.com', 27017) side of the tunnel: open new channel ssh error: SSH session not active
2024-02-13 10:51:01,626| ERROR   | Could not establish connection from local ('127.0.0.1', 3892) to remote ('insight-beta-article-db.cluster-chmaiagjipqy.ap-south-1.docdb.amazonaws.com', 27017) side of the tunnel: open new channel ssh error: SSH session not active
2024-02-13 10:51:11,691| ERROR   | Could not establish connection from local ('127.0.0.1', 3892) to remote ('insight-beta-article-db.cluster-chmaiagjipqy.ap-south-1.docdb.amazonaws.com', 27017) side of the tunnel: open new channel ssh error: SSH session not active


In [16]:
custom_umap_model = UMAP(n_neighbors=32,
                                 n_components=8,
                                 metric='cosine',
                                 random_state=86)

2024-02-13 10:51:21,765| ERROR   | Could not establish connection from local ('127.0.0.1', 3892) to remote ('insight-beta-article-db.cluster-chmaiagjipqy.ap-south-1.docdb.amazonaws.com', 27017) side of the tunnel: open new channel ssh error: SSH session not active
2024-02-13 10:51:31,829| ERROR   | Could not establish connection from local ('127.0.0.1', 3892) to remote ('insight-beta-article-db.cluster-chmaiagjipqy.ap-south-1.docdb.amazonaws.com', 27017) side of the tunnel: open new channel ssh error: SSH session not active


In [21]:
from sklearn.metrics.pairwise import pairwise_distances

In [23]:
custom_hdbscan_model = HDBSCAN(min_cluster_size=2,
                               min_samples=2,
                               metric='cosine',
                               prediction_data=True)

In [24]:
vectorizer_model = CountVectorizer(stop_words="english",
                                   max_df=0.7,
                                   ngram_range=(1, 2))

2024-02-13 10:53:12,556| ERROR   | Could not establish connection from local ('127.0.0.1', 3892) to remote ('insight-beta-article-db.cluster-chmaiagjipqy.ap-south-1.docdb.amazonaws.com', 27017) side of the tunnel: open new channel ssh error: SSH session not active
2024-02-13 10:53:22,626| ERROR   | Could not establish connection from local ('127.0.0.1', 3892) to remote ('insight-beta-article-db.cluster-chmaiagjipqy.ap-south-1.docdb.amazonaws.com', 27017) side of the tunnel: open new channel ssh error: SSH session not active
2024-02-13 10:53:32,689| ERROR   | Could not establish connection from local ('127.0.0.1', 3892) to remote ('insight-beta-article-db.cluster-chmaiagjipqy.ap-south-1.docdb.amazonaws.com', 27017) side of the tunnel: open new channel ssh error: SSH session not active


In [26]:
from sentence_transformers import SentenceTransformer

In [27]:
bertopic_model = BERTopic(umap_model=custom_umap_model,
                          hdbscan_model=custom_hdbscan_model,
                          vectorizer_model=vectorizer_model,
                          embedding_model=SentenceTransformer('BAAI/bge-large-en-v1.5'),
                          verbose=True,
                          calculate_probabilities=True)

2024-02-13 10:53:52,844| ERROR   | Could not establish connection from local ('127.0.0.1', 3892) to remote ('insight-beta-article-db.cluster-chmaiagjipqy.ap-south-1.docdb.amazonaws.com', 27017) side of the tunnel: open new channel ssh error: SSH session not active
2024-02-13 10:54:02,919| ERROR   | Could not establish connection from local ('127.0.0.1', 3892) to remote ('insight-beta-article-db.cluster-chmaiagjipqy.ap-south-1.docdb.amazonaws.com', 27017) side of the tunnel: open new channel ssh error: SSH session not active
2024-02-13 10:54:12,996| ERROR   | Could not establish connection from local ('127.0.0.1', 3892) to remote ('insight-beta-article-db.cluster-chmaiagjipqy.ap-south-1.docdb.amazonaws.com', 27017) side of the tunnel: open new channel ssh error: SSH session not active
2024-02-13 10:54:23,060| ERROR   | Could not establish connection from local ('127.0.0.1', 3892) to remote ('insight-beta-article-db.cluster-chmaiagjipqy.ap-south-1.docdb.amazonaws.com', 27017) side of the